# Hyperparameter Optimization for Graph Neural Networks

This notebook performs systematic hyperparameter optimization for graph-based EEG classification models.

## Optimization Strategy

### 1. Graph Construction Parameters
- **Connectivity Thresholds**: Optimize edge pruning thresholds
- **Node Features**: Select optimal feature representations
- **Graph Pooling**: Compare different pooling strategies

### 2. Model Architecture Parameters
- **Hidden Layer Dimensions**: Optimize GNN layer sizes
- **Number of Layers**: Find optimal network depth
- **Activation Functions**: Compare different activations
- **Dropout Rates**: Optimize regularization strength

### 3. Training Parameters
- **Learning Rate**: Adaptive learning rate schedules
- **Batch Size**: Memory-performance trade-offs
- **Optimizer Settings**: Adam vs SGD parameter tuning
- **Weight Decay**: L2 regularization optimization

## Search Methods
- **Grid Search**: Systematic parameter exploration
- **Random Search**: Efficient sampling strategy
- **Bayesian Optimization**: Advanced parameter search
- **Cross-Validation**: Robust performance estimation

## Evaluation Metrics
- **Accuracy**: Overall classification performance
- **F1-Score**: Balanced precision-recall measure
- **ROC-AUC**: Discriminative ability assessment
- **Training Efficiency**: Computational cost analysis

## Output
- Optimal hyperparameter configurations
- Performance comparison across parameter settings
- Statistical significance analysis
- Best model recommendations for deployment

In [ ]:
# Install required packages for graph neural networks
!pip install networkx torch-geometric

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import networkx as nx
import torch
import os
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# PyTorch Geometric for graph neural networks
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

In [ ]:
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
from sklearn.model_selection import train_test_split
import pandas as pd

# First, let's identify the unique groups in the DataFrame
groups = subjects_info['Group'].unique()
groups=["A","C"]
# Now, let's split each group individually
train_dfs = []
test_dfs = []
total=[]
for group in groups:
    # Filter the DataFrame for the current group
    group_df = subjects_info[subjects_info['Group'] == group]
    
    # Split the group data into training and testing sets while maintaining balance in gender
    train_group, test_group = train_test_split(group_df, test_size=0.3, stratify=group_df['Gender'], random_state=42)
    total.append(group_df)
    # Append the split data to the lists
    train_dfs.append(train_group)
    test_dfs.append(test_group)

# Concatenate the training and testing DataFrames for all groups
train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)
total_df=pd.concat(total)
# Now, train_df and test_df contain the split data with balanced groups and secondary balance in gender
# Extracting subject IDs from the training set
training_subjects = train_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Extracting subject IDs from the testing set
testing_subjects = test_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

total_subjects = total_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Displaying the lists of subjects
print("Training Subjects:")
print(training_subjects)

print("\nTesting Subjects:")
print(testing_subjects)

print("\nTotal Subjects:")
print(total_subjects)

In [ ]:
import pandas as pd
import networkx as nx
import os
from tqdm import tqdm

# Directory paths
pairwise_features_dir = '/kaggle/input/new-extraction/eeg_connectivity_features/'
node_features_dir = '/kaggle/input/new-extraction/node_features/'

# Function to load data
def load_data(subject_id):
    pairwise_path = os.path.join(pairwise_features_dir, f'eeg_connectivity_features_subject_{subject_id}.csv')
    node_path = os.path.join(node_features_dir, f'node_features_subject_{subject_id}.csv')
    
    pairwise_df = pd.read_csv(pairwise_path)
    node_df = pd.read_csv(node_path)
    
    return pairwise_df, node_df

# Function to create graph from data
def create_graph(pairwise_df, node_df, sample):
    G = nx.Graph()
    
    # Add nodes with attributes from node_df
    for _, row in node_df[node_df['sample'] == sample].iterrows():
        G.add_node(row['channel'], **{
            'psd_mean': row['psd_mean'],
            'psd_std': row['psd_std'],
            'entropy': row['entropy'],
            'hjorth_activity': row['hjorth_activity'],
            'hjorth_mobility': row['hjorth_mobility'],
            'hjorth_complexity': row['hjorth_complexity'],
            'spectral_entropy': row['spectral_entropy']
        })
    
    # Add edges with attributes from pairwise_df
    for _, row in pairwise_df[pairwise_df['sample'] == sample].iterrows():
        G.add_edge(row['channel_1'], row['channel_2'], **{
            
            'plv': row['plv'],
            'mutual_information': row['mutual_information'],
            'pearson_correlation': row['pearson_correlation']
        })
    
    return G

# Process subjects

# Optionally save graphs or process them for a graph neural network framework

# Example: Converting to PyTorch Geometric format
from torch_geometric.data import Data
import torch

def nx_to_pyg_data(G,label):
    # Convert NetworkX graph to PyTorch Geometric Data
    edge_index = torch.tensor(list(G.edges)).t().contiguous().long()
    
    # Node features
    x = []
    for _, data in G.nodes(data=True):
        x.append([data['psd_mean'], data['psd_std'], data['entropy'], data['hjorth_activity'],
                  data['hjorth_mobility'], data['hjorth_complexity'], data['spectral_entropy']])
    x = torch.tensor(x, dtype=torch.float)
    
    # Edge features
    edge_attr = []
    for _, _, data in G.edges(data=True):
        edge_attr.append([ data['plv'], data['mutual_information'], data['pearson_correlation']])
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr,y=torch.tensor([label], dtype=torch.float))
    return data



In [ ]:
all_graphs = []
groups = []
count = 0
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
labels = []

for subject_id in tqdm(total_subjects):
    pairwise_df, node_df = load_data(subject_id)
    samples = pairwise_df['sample'].unique()
    count += len(samples)
    num_epochs = len(samples)  # Number of epochs, replace 'your_data_key' as before
    
    subject_id = f"sub-{str(subject_id).zfill(3)}"  # Format subject ID
    group_info = subjects_info[subjects_info['participant_id'] == subject_id]['Group'].values[0]
    duplicated_groups = [group_info] * num_epochs
    labels.extend(duplicated_groups)

    groups.extend([subject_id] * num_epochs)
    
    for sample in samples:
        G = create_graph(pairwise_df, node_df, sample)
        all_graphs.append(G)
        
        


In [ ]:
# Assuming you define label_mapping, encoder, and encode labels as in your code
#####################################################
label_mapping = {'A': 0, 'C': 1, 'F': 2}  # You can adjust this mapping as needed
#####################################################
df=pd.Series(labels).map(label_mapping)

encoder = OneHotEncoder(categories='auto', sparse=False)
encoder.fit(np.array(pd.Series(labels).map(label_mapping)).reshape(-1, 1))
# Encode labels
################################################

total_labels_encoded = encoder.transform(np.array(pd.Series(labels).map(label_mapping).values).reshape(-1, 1))
################################################


print("Total labels shape:", total_labels_encoded.shape)

In [ ]:
pyg_graphs = [nx_to_pyg_data(G,labels) for G, labels in zip(all_graphs, total_labels_encoded)]
print(len(pyg_graphs))

## the model 

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU
from torch_geometric.nn import NNConv, global_mean_pool

class GNN(torch.nn.Module):
    def __init__(self, in_channels, edge_in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        # Define the network for edge features
        nn = Sequential(
            Linear(edge_in_channels, 25),
            ReLU(),
            Linear(25, in_channels * hidden_channels)
        )
        self.conv1 = NNConv(in_channels, hidden_channels, nn, aggr='mean')
        
        # Define another network for the second NNConv layer if needed
        nn2 = Sequential(
            Linear(edge_in_channels, 25),
            ReLU(),
            Linear(25, hidden_channels * hidden_channels)
        )
        self.conv2 = NNConv(hidden_channels, hidden_channels, nn2, aggr='mean')
        
        self.fc = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_attr, batch):
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = global_mean_pool(x, batch)  # Global pooling
        x = self.fc(x)
        return F.log_softmax(x, dim=1)



In [ ]:
import optuna
import torch
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from torch_geometric.data import Data, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

batch_size = 128
num_epochs = 500
patience = 50  # Number of epochs to wait before early stopping
labels_total = np.array(total_labels_encoded)  # Convert to numpy array
binary_labels = np.any(labels_total, axis=1).astype(int)  # Convert to binary (0 or 1)

def objective(trial):
    fold = 1
    group_indices = skf.split(np.arange(len(pyg_graphs)), groups)

    total_y_true = []
    total_y_pred = []

    train_losses = []
    val_losses = []

    hidden_channels = trial.suggest_int("hidden_channels", 16, 128)
    learning_rate = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    
    for train_indices, test_indices in group_indices:
        print(f"Fold {fold}/5")
        
        # Split the dataset into train and test sets based on indices
        train_data = [pyg_graphs[i] for i in train_indices]
        test_data = [pyg_graphs[i] for i in test_indices]
        
        # Create DataLoader for train and test sets
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

        model = GNN(in_channels=7, edge_in_channels=3, hidden_channels=hidden_channels, out_channels=2).to(device)
        
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        best_val_loss = float('inf')
        early_stop_counter = 0
        best_model_state = None  # Variable to store the best model's state

        # Training loop
        train_loss_list = []
        val_loss_list = []

        for epoch in range(num_epochs):
            running_loss = 0.0

            for batch in train_loader:
                optimizer.zero_grad()
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            # Calculate validation loss
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for batch in test_loader:
                    batch = batch.to(device)
                    out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                    loss = criterion(out, batch.y)
                    val_loss += loss.item()
            val_loss /= len(test_loader) + 0.001
            print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}")

            train_loss_list.append(running_loss/len(train_loader))
            val_loss_list.append(val_loss)

            # Early stopping and save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict()  # Save the best model's state
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if early_stop_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Load the best model's state
        model.load_state_dict(best_model_state)

        # Evaluate on the test set
        model.eval()
        y_true = []
        y_pred = []
        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                _, predicted = torch.max(out, 1)
                y_true.extend(batch.y.cpu().tolist())
                y_pred.extend(predicted.cpu().tolist())
                total_y_true.extend(batch.y.cpu().tolist())
                total_y_pred.extend(predicted.cpu().tolist())

        # Calculate classification metrics
        acc = accuracy_score(np.argmax(y_true, axis=1), y_pred)
        sens = recall_score(np.argmax(y_true, axis=1), y_pred, average='binary', pos_label=1)
        spec = recall_score(np.argmax(y_true, axis=1), y_pred, average='binary', pos_label=0)
        prec = precision_score(np.argmax(y_true, axis=1), y_pred, average='binary', pos_label=1)
        f1 = f1_score(np.argmax(y_true, axis=1), y_pred, average='binary', pos_label=1)
        recall = recall_score(np.argmax(y_true, axis=1), y_pred, average='binary', pos_label=1)
        confusion_mat = confusion_matrix(np.argmax(y_true, axis=1), y_pred)

        print(f"Fold {fold}/5 Classification Metrics:")
        print(f"Accuracy: {acc:.4f}")
        print(f"Sensitivity: {sens:.4f}")
        print(f"Specificity: {spec:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"Confusion Matrix:\n{confusion_mat}")
        print("\n")

        # Plot training curves for the current fold
        plt.figure()
        plt.plot(range(len(train_loss_list)), train_loss_list, label='Train Loss')
        plt.plot(range(len(val_loss_list)), val_loss_list, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Training Curves for Fold {fold}')
        plt.legend()
        plt.show()

        train_losses.extend(train_loss_list)
        val_losses.extend(val_loss_list)

        fold += 1

    return best_val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print(f"Best trial:\n{study.best_trial}")
print(f"Best hyperparameters: {study.best_trial.params}")
